This tutorial generates a simulated data and use EBtimecourse to find change points in the data.

In [5]:
rm(list=ls())
setwd("~/Documents/Research/Microarray_time_course/manuscript/EBtimecourse")
library(tensorflow)
use_python("/usr/local/bin/python3")
source("EBtimecourse.R")

Use Normal Normal-Gamma model to generate a data matrix of 1000 genes by 8 time points, and numbers of replicates for the 8 time points are 3, 3, 3, 4, 2, 3, 3 respectively, so total columns are 24. In this simulation, we set P=0.2, which means 20% genes having change points and these genes are put in the top 20% rows of the data matrix. In the parameter matrix, it stores parameters to generate data. In the parameter matrix, mu1 and mu2 stand for the latent means, sigma1 and sigma2 stand for variances. n1, n2 and n3 stand for the length of the three homogeneous sequences, for example, if n1=2, n2=1, n3=5, it means the two change points are after the 2nd and 3rd time point.

set.seed(0)

P=0.2 # proportion of genes to have change points
N=1000;

nDE=P*N; nEE=N-nDE;
cp_real_seq_index=1:nDE;
timePoint=8; # number of time points
replicate=c(3,3,3,4,2,3,3,3); # number of replicated of each time points, for example, this vector means 4th time point has 4 replicates and 5th time point has 2 replicates, the others have 3 replicates
Ti=sum(replicate); 

changePointTable = data.frame(matrix(NA, nrow=(timePoint-1)+(timePoint-1)*(timePoint-2)/2, ncol=3), stringsAsFactors=F)
colnames(changePointTable) = c("n1", "n2", "n3")
changePointTable[1:(timePoint-1),"n1"]=1:(timePoint-1)
changePointTable[1:(timePoint-1),"n2"]=(timePoint-1):1
changePointTable[1:(timePoint-1),"n3"]=0
combT = as.data.frame(t(combn(timePoint-1, 2)))
combT$V2 = combT$V2 - combT$V1
changePointTable[timePoint:nrow(changePointTable),c("n1","n2")]=combT
changePointTable[timePoint:nrow(changePointTable),"n3"]=timePoint-rowSums(changePointTable[timePoint:nrow(changePointTable),c("n1", "n2")])
combNumber = nrow(changePointTable)

ss=sample(1:nrow(changePointTable), P*N, replace=T)
n1=changePointTable[ss,"n1"]
n2=changePointTable[ss,"n2"]
n3=changePointTable[ss,"n3"]

n1=c(n1, rep(timePoint,nEE)); n2=c(n2, rep(0, nEE)); n3=c(n3, rep(0, nEE));
mu0=0; kappa0=0.1; alpha0=1; beta0=10;
lambda = rgamma(N*2, shape=alpha0, rate=beta0)
mu = rnorm(N*2,mean=mu0,sd=1/sqrt(kappa0*lambda))
lambda = matrix(lambda, ncol=2)
mu = matrix(mu, ncol=2)
sigma = 1/sqrt(lambda)
parameter=data.frame(mu, sigma, n1=n1, n2=n2, n3=n3)
colnames(parameter)=c("mu1", "mu2", "sigma1", "sigma2", "n1", "n2", "n3")

gene.de=t(apply(parameter[1:nDE,],1, function(x) 
  c(rnorm(sum(replicate[1:x[5]]), mean=x[1], sd=x[3]), rnorm(sum(replicate[(x[5]+1):(x[5]+x[6])]), mean=x[2], sd=x[4]),
    rnorm(sum(replicate[-c(1:(x[5]+x[6]))]), mean=x[1], sd=x[3]))))
gene.ee=t(apply(parameter[(nDE+1):N,],1, function(x) c(rnorm(Ti, mean=x[1], sd=x[3]))))

gene.exp=rbind(gene.de, gene.ee)
                
dim(gene.exp)
head(gene.exp)
head(parameter)

In [ ]:
Run EBtimecourse

In [7]:
ptm <- proc.time()
result = EBtimecourse(exp.dat = gene.exp, timepoint = timePoint, replicate = replicate, FDR=0.1, verbose=T)
print(proc.time() - ptm)

1 84437.506638011

Max delta ll: 3.12596615462098

21 84381.4291577851

Max delta ll: 3.09676710065105

41 84337.0483235767

Max delta ll: 2.47641555481823

61 84300.7773852293

Max delta ll: 1.97489969374146

81 84270.1536760259

Max delta ll: 1.65300675845356

101 84243.7933529963

Max delta ll: 1.40883947689144

121 84220.6307683346

Max delta ll: 1.22671443571744

141 84199.8958695307

Max delta ll: 1.08895224895969

161 84181.0167358295

Max delta ll: 0.983958973331028

181 84163.5634018876

Max delta ll: 0.903447514443542

201 84147.2092095706

Max delta ll: 0.841462130978471

221 84131.704192108

Max delta ll: 0.793613689864287

241 84116.8562123393

Max delta ll: 0.756613586287131

261 84102.5171955477

Max delta ll: 0.727966624210239

281 84088.5728978882

Max delta ll: 0.705761939636432

301 84074.9351529759

Max delta ll: 0.688529405189911

321 84061.5359431894

Max delta ll: 0.675133223965531

341 84048.3228133488

Max delta ll: 0.664696234380244

361 84035.2553143558

Max 

2961 82128.9947398685

Max delta ll: 0.92557627025235

2981 82110.4401294924

Max delta ll: 0.929684661561623

3001 82091.8030143448

Max delta ll: 0.933825204119785

3021 82073.0827472501

Max delta ll: 0.937998295383295

3041 82054.2786731697

Max delta ll: 0.942204325110652

3061 82035.3901288613

Max delta ll: 0.946443714768975

3081 82016.4164428638

Max delta ll: 0.950716875013313

3101 81997.3569353303

Max delta ll: 0.955024231443531

3121 81978.2109179144

Max delta ll: 0.959366206036066

3141 81958.9776934989

Max delta ll: 0.963743249492836

3161 81939.6565561282

Max delta ll: 0.968155801296234

3181 81920.2467908178

Max delta ll: 0.972604320530081

3201 81900.7476733913

Max delta ll: 0.977089272229932

3221 81881.1584703442

Max delta ll: 0.98161111465015

3241 81861.4784387371

Max delta ll: 0.98617033867049

3261 81841.7068258346

Max delta ll: 0.99076743455953

3281 81821.8428689601

Max delta ll: 0.995402902801288

3301 81801.885795445

Max delta ll: 1.00007724095485

Max delta ll: 2.46758608763048

5961 77726.0772363111

Max delta ll: 2.4946033602464

5981 77675.8965927643

Max delta ll: 2.52217566594481

6001 77625.1585680929

Max delta ll: 2.5503151112498

6021 77573.8516943935

Max delta ll: 2.5790335679485

6041 77521.9642695909

Max delta ll: 2.60834257994429

6061 77469.4843651937

Max delta ll: 2.63825324612844

6081 77416.399836622

Max delta ll: 2.66877607845527

6101 77362.6983366735

Max delta ll: 2.69992083843681

6121 77308.3673325438

Max delta ll: 2.73169633223733

6141 77253.394127462

Max delta ll: 2.76411016692873

6161 77197.7658877239

Max delta ll: 2.79716844754876

6181 77141.469676445

Max delta ll: 2.83087541314308

6201 77084.4924955019

Max delta ll: 2.86523298433167

6221 77026.8213373774

Max delta ll: 2.90024022034777

6241 76968.4432492508

Max delta ll: 2.93589264736511

6261 76909.3454119314

Max delta ll: 2.97218144351791

6281 76849.5152369364

Max delta ll: 3.00909244595096

6301 76788.9404855959

Max delta ll: 3.

8821 74654.9560551075

Max delta ll: 7.63591378927231e-06

8841 74654.9559081072

Max delta ll: 7.44144199416041e-06

8861 74654.9557649258

Max delta ll: 7.2493712650612e-06

8881 74654.9556255143

Max delta ll: 7.05971615388989e-06

8901 74654.9554898228

Max delta ll: 6.87252031639218e-06

8921 74654.9553578005

Max delta ll: 6.68778375256807e-06

8941 74654.9552293961

Max delta ll: 6.50566653348505e-06

8961 74654.955104557

Max delta ll: 6.32621231488883e-06

8981 74654.9549832304

Max delta ll: 6.14931923337281e-06

9001 74654.9548653625

Max delta ll: 5.97511825617403e-06

9021 74654.9547508991

Max delta ll: 5.80353662371635e-06

9041 74654.9546397851

Max delta ll: 5.6347344070673e-06

9061 74654.9545319652

Max delta ll: 5.468784365803e-06

9081 74654.9544273834

Max delta ll: 5.3054973250255e-06

9101 74654.9543259834

Max delta ll: 5.14510611537844e-06

9121 74654.9542277082

Max delta ll: 4.98749432154e-06

9141 74654.9541325009

Max delta ll: 4.83274925500154e-06

9161 7

Max delta ll: 7.8580342233181e-10

11601 74654.95190756

Max delta ll: 6.69388100504875e-10

11621 74654.9519075495

Max delta ll: 5.67524693906307e-10

11641 74654.9519075404

Max delta ll: 5.0931703299284e-10

11661 74654.9519075324

Max delta ll: 4.36557456851006e-10

11681 74654.9519075255

Max delta ll: 3.92901711165905e-10

11701 74654.9519075195

Max delta ll: 3.34694050252438e-10

11721 74654.9519075143

Max delta ll: 2.91038304567337e-10

11741 74654.9519075099

Max delta ll: 2.61934474110603e-10

11761 74654.951907506

Max delta ll: 2.47382558882236e-10

11781 74654.9519075027

Max delta ll: 2.03726813197136e-10

11801 74654.9519074998

Max delta ll: 1.89174897968769e-10

11821 74654.9519074974

Max delta ll: 1.74622982740402e-10

11841 74654.9519074952

Max delta ll: 1.45519152283669e-10

11861 74654.9519074935

Max delta ll: 1.16415321826935e-10

11881 74654.951907492

Max delta ll: 1.30967237055302e-10

11901 74654.9519074906

Max delta ll: 1.01863406598568e-10

11904 7465

[1] "Converge params:"
$P
[1] 0.8041634

$mu0
[1] -0.22962

$kappa0
[1] 0.1008914

$alpha0
[1] 0.9621105

$beta0
[1] 9.723646

   user  system elapsed 
153.157  38.027  58.215 


Summarize the sensitivity and FDR for Q1 and Q2

In [10]:
Q1_sensitivity = sum(result$cp.index %in% cp_real_seq_index)/nDE
Q1_FDR = sum(!(result$cp.index %in% cp_real_seq_index))/length(result$cp.index)
cp_position_correct_numer = 0
cp_position_not_correct_numer = 0
for(i in 1:nrow(result$cp.position)) {
  if(parameter[result$cp.position$SeqID[i],"n1"]==result$cp.position[i, "n1"] & parameter[result$cp.position$SeqID[i],"n2"]==result$cp.position[i, "n2"])
    cp_position_correct_numer=cp_position_correct_numer+1
  else
    cp_position_not_correct_numer = cp_position_not_correct_numer+1
}
Q2_sensitivity=cp_position_correct_numer/nDE
Q2_FDR=cp_position_not_correct_numer/nrow(result$cp.position)

print(Q1_sensitivity)
print(Q1_FDR)
print(Q2_sensitivity)
print(Q2_FDR)

[1] 0.795
[1] 0.0755814
[1] 0.725
[1] 0.07643312
